# Neural net

In [1]:
import pandas as pd
import numpy as np
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, Dropout, Bidirectional, Embedding, LSTM, GRU, BatchNormalization, TextVectorization, Flatten
from tensorflow.keras.optimizers import Adam
genomes = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="genomics_cannabis")

To swiftly go through the dataset:

In [2]:
bq_assistant = BigQueryHelper("bigquery-public-data", "genomics_cannabis")
bq_assistant.list_tables()

['MNPR01_201703',
 'MNPR01_reference_201703',
 'MNPR01_transcriptome_201703',
 'cs10_gff',
 'cs3k_project_info',
 'cs3k_vcf_cs10_dv090',
 'sample_info_201703']

In [3]:
query = """SELECT
  variant_id, alternate_bases, quality, type, 
  AB, ABP, AC, AF, AN, AO, DP, DPB, DPRA, EPP, 
  EPPR, GTI, MQM, MQMR, NS, NUMALT, ODDS, PAO,
  PQA, PQR, PRO, QA, QR, RO, RPL, RPP, RPPR, 
  RPR, RUN, SAF, SAP, SAR, SRF, SRP, SRR
FROM
  `bigquery-public-data.genomics_cannabis.MNPR01_201703` v
limit 500000;"""
response = genomes.query_to_pandas_safe(query, max_gb_scanned=100)
response.head(10)

C:\Users\sharm\AppData\Roaming\Python\Python311\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,variant_id,alternate_bases,quality,type,AB,ABP,AC,AF,AN,AO,...,RPP,RPPR,RPR,RUN,SAF,SAP,SAR,SRF,SRP,SRR
0,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTI1NzF8Z2J8TU5QUj...,[CTA],23.39620,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
1,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTEyODV8Z2J8TU5QUj...,[CGTAA],7.52241,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
2,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY5MTJ8Z2J8TU5QUj...,[CCTT],25.30020,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
3,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY5OTl8Z2J8TU5QUj...,[TATGG],6.16805,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
4,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODY2OTF8Z2J8TU5QUj...,[ATTTTTTTAA],3.08123,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
5,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTQyOTF8Z2J8TU5QUj...,[GAG],23.56570,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
6,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTUwNTV8Z2J8TU5QUj...,[GC],27.00690,[del],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
7,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTUyMDF8Z2J8TU5QUj...,[GATAAA],14.01670,[mnp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
8,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTU1NTV8Z2J8TU5QUj...,[TTTTGC],12.62600,[mnp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[1],[5.18177],[0],0,0.0,0
9,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODgxOTZ8Z2J8TU5QUj...,[AGT],9.12466,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[1],[1],[0],[5.18177],[1],0,0.0,0


To process the DNA strands:

In [4]:
response["DNA_A"] = response["alternate_bases"].str[0].str.count('A')
response["DNA_C"] = response["alternate_bases"].str[0].str.count('C')
response["DNA_G"] = response["alternate_bases"].str[0].str.count('G')
response["DNA_T"] = response["alternate_bases"].str[0].str.count('T')

To correct protein data:

In [5]:
proteins = ["AB", "ABP", "AC", "AF", "AO", "DPRA", "EPP", "MQM", "PAO",
            "PQA", "QA", "RPL", "RPP", "type",
            "RPR", "RUN", "SAF", "SAP", "SAR", 
]

for protein in proteins:
    response[protein] = response[protein].str[0]

To dummify the types:

In [6]:
dummies = pd.get_dummies(response["type"], dtype="int")

Here, I shall define my X and y. 

In [7]:
X = pd.concat([
    response[["AB", "ABP", "AC", "AF", "AN", "AO", "DP", "DPB", "DPRA", "EPP", 
              "EPPR", "GTI", "MQM", "MQMR", "NS", "NUMALT", "ODDS", "PAO",
              "PQA", "PQR", "PRO", "QA", "QR", "RO", "RPL", "RPP", "RPPR", 
              "RPR", "RUN", "SAF", "SAP", "SAR", "SRF", "SRP", "SRR"
    ]], 
    dummies
], axis=1)
y = response[["DNA_A", "DNA_C", "DNA_G", "DNA_T"]]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import torch
import torch.nn as nn

from tqdm.notebook import tnrange, tqdm_notebook

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=1337)

In [10]:
X.shape

(500000, 40)

In [11]:
X_train

,AB,ABP,AC,AF,AN,AO,DP,DPB,DPRA,EPP,...,SAP,SAR,SRF,SRP,SRR,complex,del,ins,mnp,snp
488195,0.0,0.0,2,1.0,2,3,3,3.0000,0,9.52472,...,9.52472,3,0,0.0,0,1,0,0,0,0
206837,0.0,0.0,2,1.0,2,4,4,4.0000,0,11.69620,...,11.69620,4,0,0.0,0,0,0,0,0,1
45480,0.0,0.0,2,1.0,2,7,8,8.0000,0,3.32051,...,5.80219,2,0,0.0,0,0,0,0,0,1
179556,0.0,0.0,2,1.0,2,2,2,3.0000,0,7.35324,...,7.35324,0,0,0.0,0,0,1,0,0,0
136061,0.0,0.0,2,1.0,2,3,3,3.0000,0,3.73412,...,3.73412,2,0,0.0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216231,0.0,0.0,2,1.0,2,7,8,8.0000,0,3.32051,...,18.21060,7,0,0.0,0,0,0,0,1,0
271592,0.0,0.0,2,1.0,2,6,15,16.5714,0,3.01030,...,16.03910,6,0,0.0,0,1,0,0,0,0
492380,0.0,0.0,2,1.0,2,4,4,4.0000,0,5.18177,...,11.69620,0,0,0.0,0,0,0,0,1,0
450749,0.0,0.0,2,1.0,2,14,14,14.0000,0,3.63072,...,33.41090,14,0,0.0,0,0,0,0,0,1


In [12]:
X_train = torch.tensor(X_train.to_numpy(dtype='float32'), dtype=torch.float32)
X_test = torch.tensor(X_test.to_numpy(dtype='float32'), dtype=torch.float32)

y_train = torch.tensor(y_train.to_numpy(dtype='float32'), dtype=torch.long)
y_test = torch.tensor(y_test.to_numpy(dtype='float32'), dtype=torch.long)

In [14]:
x = X_train[0:8, :]
target = y_train[0:8].to(torch.float32)

neural_net = nn.Sequential(nn.Linear(40, 4),
                           nn.ReLU(),
                           nn.Linear(4, 1))
mse_loss = nn.MSELoss()

optimizer = torch.optim.SGD(neural_net.parameters(), lr=0.1)

output_values = neural_net(x)
loss = mse_loss(output_values, target)

for _ in range(0, 1000):
    loss.backward()
    optimizer.step()

    new_output = neural_net(x)
    new_loss = mse_loss(new_output, target)
    print(f"Old loss: {loss}; New loss: {new_loss}")
    loss = new_loss

Old loss: 4.424934387207031; New loss: 0.8009885549545288
Old loss: 0.8009885549545288; New loss: 0.7667326927185059
Old loss: 0.7667326927185059; New loss: 1.0242927074432373
Old loss: 1.0242927074432373; New loss: 1.3779230117797852
Old loss: 1.3779230117797852; New loss: 1.5588645935058594
Old loss: 1.5588645935058594; New loss: 1.429602026939392
Old loss: 1.429602026939392; New loss: 1.0883746147155762
Old loss: 1.0883746147155762; New loss: 0.7945152521133423
Old loss: 0.7945152521133423; New loss: 0.7713571786880493
Old loss: 0.7713571786880493; New loss: 1.0365004539489746
Old loss: 1.0365004539489746; New loss: 1.388435959815979
Old loss: 1.388435959815979; New loss: 1.5596930980682373
Old loss: 1.5596930980682373; New loss: 1.4201159477233887
Old loss: 1.4201159477233887; New loss: 1.0757834911346436
Old loss: 1.0757834911346436; New loss: 0.7883883118629456
Old loss: 0.7883883118629456; New loss: 0.7763506174087524
Old loss: 0.7763506174087524; New loss: 1.0488190650939941
Ol

This is a low amount of loss. 